In [38]:
import pandas as pd
import numpy as np

In [39]:
# this should probably be on a seperate cell so I don't constantly reload the dataframes
# constants so I don't have to keep changing names
BS = 'business'
CH = 'checkin'
TI = 'tip'
RW = 'review'
US = 'user'

# load files in (right now I'm skipping the reviews part cause it's big and my work laptop doens't have the space)
fDefPath = 'reviews/yelpReviews/yelp_academic_dataset_'
fYelpNames = [BS, CH, TI]


In [40]:
fdf = {}

# load the datasets
for name in fYelpNames:
    fPath = f'{fDefPath}{name}.json'
    fdf[name] = pd.read_json(fPath, lines=True)

for name in fYelpNames:
    print(fdf[name].columns)

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')
Index(['business_id', 'date'], dtype='object')
Index(['user_id', 'business_id', 'text', 'date', 'compliment_count'], dtype='object')


In [47]:
bsdf = fdf[BS].dropna(subset=["categories"]).copy()
bsdf.loc[:,"categoryList"] = bsdf.loc[:,"categories"].str.split(", ")

# Flatten and count unique categories
allCategories = bsdf["categoryList"].explode().str.lower()
categoryCounts = allCategories.value_counts()

# Preview common categories
print(categoryCounts.head(50))

categoryList
restaurants                  52268
food                         27781
shopping                     24395
home services                14356
beauty & spas                14292
nightlife                    12281
health & medical             11890
local services               11198
bars                         11065
automotive                   10773
event planning & services     9895
sandwiches                    8366
american (traditional)        8139
active life                   7687
pizza                         7093
coffee & tea                  6703
fast food                     6472
breakfast & brunch            6239
american (new)                6097
hotels & travel               5857
home & garden                 5799
fashion                       5739
burgers                       5636
arts & entertainment          5434
auto repair                   5433
hair salons                   5046
nail salons                   4621
mexican                       4600
italian

In [50]:
# finding what american new means
bsdf[bsdf["categories"].str.contains(r"American \(New\)", na=False)].name.head(50)


58                                      Roman Forum
61                                       Twin Peaks
64       Doc Magrogan's Oyster House - West Chester
79                          Top Shelf Sports Lounge
99                           Ricardo's Italian Cafe
110                                Sunset 44 Bistro
181     Gaylord Opryland Resort & Convention Center
198                                    Aussie Grill
231                     Eat Mah Taco @ Pal's Lounge
247                        The Libertine Liquor Bar
372                                        The Love
386                                Ryan Christopher
443                                The Twisted Tail
461                                     Local Links
476                                    Garage Pizza
502                            NY Bagel Cafe & Deli
524                                Whetstone Tavern
537                                      The Bricks
557                             Reno Downtown Joint
559         

In [51]:
# create a manual category for now
categoryMap = {
    'restaurants' : {'restaurants', 'food', 'sandwiches', 'american (new)', 'american (traditional)'},
    'retail' : {'shopping', 'home & garden', 'fashion', 'real estate', 'grocery'},
    'beauty' : {'beauty & spas', 'hair salons', 'nail salons'},
    'auto' : {'automotive', 'auto repair'},
    'services' : {'home services', 'local services', 'event planning & services'},
    'fitness' : {'active life', 'fitness & instruction'},
    'health' : {'health & medical', 'doctors', 'dentist'},
    'nightlife' : {'nightlife', 'bars'},
    'lodging' : {'hotesl & travel', 'hotels'}
}